In [6]:
import pandas as pd
import numpy as np

# web scraper

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import lxml.html
import re
import time

# write to csv file

import csv

# joblib is a set of tools to provide lightweight pipelining in Python. It provides utilities for saving and loading Python objects that make use of NumPy data structures, efficiently.
import joblib



# -------------------------------- Web Scrapper ---------------------------

url = 'https://www.amazon.com/events/epicdeals?ref_=nav_cs_gb_5d8ef71f209b44c2acab6343d6a5a05e'
#url='https://www.currys.ie/ieen/search-keywords/xx_xx_xx_xx_xx/-wk22_headphones_ie-/xx-criteria.html'

# to avoid opening browser while using selenium
option = webdriver.ChromeOptions()
option.add_argument('headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options=option)

driver.get(url)
time.sleep(3)

# get source code -- type: str
html_source = driver.page_source

# key
html = lxml.html.fromstring(html_source)

# obtain all the text under the 'div' tags
items = html.xpath("//text()")

pattern = re.compile("^\s+|\s+$|\n")

clause_text = ""

for item in items:
    line = re.sub(pattern, "", item)
    if len(item) > 1:
        clause_text += line +"\n"

driver.quit()



# -------------------------------- Scrapping Dataset ---------------------------

raw_text = clause_text

# the beginning character of the content, which is the sign we should ignore the content
ignore_str = ',.;{}?#/!'

# the content we are going to keep to send to models.
content_list = []

# only keep the content that has words count from 2 to 20 (includes).
for line in raw_text.split('\n'):
    if 1<len(line.split())<21 and line[0] not in ignore_str:
        content_list.append([line])

header = ['content']

# create a csv file to save the filtered content for later model analysis.
with open('Websites/web11.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    writer.writerows(content_list)
    
    
        
# -------------------------------- Check Presence ---------------------------    


# Loading the saved model with joblib
presence_model = joblib.load('rf_presence_classifier.joblib')
presence_cv = joblib.load('presence_CountVectorizer.joblib')

# New dataset to predict
presence_pred = pd.read_csv('Websites/web11.csv')


# Filter out the disturibing content to be removed
str_list = ['low to high','high to low','high low','low high','{','ships','ship','®',
            'limited edition','cart is currently empty','out of stock','believe in',
            'today\'s deals','customer service','offer available', '% off']
pattern = '|'.join(str_list)

presence_pred = presence_pred[~presence_pred.content.str.lower().str.contains(pattern)]



# apply the pretrained model to the new content data
pre_pred_vec = presence_model.predict(presence_cv.transform(presence_pred['content'].str.lower()))

presence_pred['presence'] = pre_pred_vec.tolist()

# dark pattern content are those where the predicted result equals to 0.
dark = presence_pred.loc[presence_pred['presence']==0]

dark.to_csv('Websites/web11-rf.csv', index = False, header = True)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/zenglan/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


In [5]:
# Loading the saved model with joblib
cat_model = joblib.load('mnb_category_classifier.joblib')
cat_cv = joblib.load('category_CountVectorizer.joblib')

# mapping of the encoded dark pattern categories.
cat_dic = {0:'Forced Action', 1:'Misdirection', 2:'Obstruction', 3:'Scarcity', 4:'Sneaking',
           5:'Social Proof', 6:'Urgency'}

# apply the model and the countvectorizer to the detected dark pattern content data
cat_pred_vec = cat_model.predict(cat_cv.transform(dark['content']))


dark['category'] = cat_pred_vec.tolist()

category_list = dark['category'].tolist()

# get the mapping of the category name and encoded category integers
dark['category_name'] = [cat_dic[int(category)] for category in category_list]

# reset the index of the detected dark pattern list on the webpage.
dark = dark.reset_index(drop=True)

dark

<ipython-input-5-de5a69d3bb62>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dark['category'] = cat_pred_vec.tolist()
<ipython-input-5-de5a69d3bb62>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dark['category_name'] = [cat_dic[int(category)] for category in category_list]


,content,presence,category,category_name
0,Get 20% off selected HP accessories when bough...,0,6,Urgency
1,Save €80 on McAfee LiveSafe Anti-virus (1 Year...,0,5,Social Proof
2,"Sorry, but right now we can only show options for",0,3,Scarcity
3,Order online to collect in store ...,0,6,Urgency
4,Standard delivery within 5-7 working days,0,6,Urgency
5,Please add this item to your basket. We'll sho...,0,1,Misdirection
6,Get 20% off selected HP accessories when bough...,0,6,Urgency
7,Save €80 on McAfee LiveSafe Anti-virus (1 Year...,0,5,Social Proof
8,Get a free upgrade to Windows 11 when available,0,6,Urgency
9,Reviewer left no comment,0,3,Scarcity
